In [1]:
# Connecting to database
import os.path
import yaml
from sqlalchemy import create_engine, text
from sqlalchemy.orm import Session

config_path = os.path.expanduser('~\\.sqlalchemy\\config.yaml')

with open(config_path) as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

db_path = (
    f"postgresql://{config['username']}:{config['password']}"
    f"@{config['host']}:{config['port']}/{config['db_name']}"
)
engine = create_engine(db_path)


In [2]:
# Creating session
session = Session(engine)


In [3]:
# Reading excel files
import pandas as pd
file_path1 = os.path.expanduser('~\\.sqlalchemy\\한국인이름.xlsx')
file_path2 = os.path.expanduser('~\\.sqlalchemy\\인구통계.xlsx')

last_name = pd.read_excel(file_path1, sheet_name=0, header=0)
first_name = pd.read_excel(file_path1, sheet_name=1, header=0)
age_stat = pd.read_excel(file_path2, sheet_name=0, header=0)
region_stat = pd.read_excel(file_path2, sheet_name=1, header=0)
sex_stat = pd.read_excel(file_path2, sheet_name=2, header=0)


In [4]:
# Creating table schema
from sqlalchemy import MetaData, Table, Column, Float, String
metadata_obj = MetaData()
lastname_table = Table(
    "korean_lastname",
    metadata_obj,
    Column('lastname', String(5), primary_key=True),
    Column('ratio', Float(8))
)

firstname_table = Table(
    "korean_firstname",
    metadata_obj,
    Column('firstname', String(5), primary_key=True),
    Column('ratio', Float(8))
)

age_table = Table(
    "age_stat",
    metadata_obj,
    Column('age', String(5), primary_key=True),
    Column('ratio', Float(8))
)

region_table = Table(
    "region_stat",
    metadata_obj,
    Column('region', String(8), primary_key=True),
    Column('ratio', Float(8))
)

sex_table = Table(
    "sex_stat",
    metadata_obj,
    Column('sex', String(2), primary_key=True),
    Column('ratio', Float(8))
)


In [5]:
# Emitting DDL to the Database
metadata_obj.create_all(engine)


In [6]:
# Setting up the Registry
from sqlalchemy.orm import registry
mapper_registry = registry()
mapper_registry.metadata
Base = mapper_registry.generate_base()


In [7]:
# Declaring Mapped Classes (using base)
class KoreanLastname(Base):
    __tablename__ = 'korean_lastname'

    lastname = Column(String(5), primary_key=True)
    ratio = Column(Float(8))


class KoreanFirstname(Base):
    __tablename__ = 'korean_firstname'

    firstname = Column(String(5), primary_key=True)
    ratio = Column(Float(8))


class AgeStat(Base):
    __tablename__ = 'age_stat'

    age = Column(String(5), primary_key=True)
    ratio = Column(Float(8))


class RegionStat(Base):
    __tablename__ = 'region_stat'

    region = Column(String(8), primary_key=True)
    ratio = Column(Float(8))


class SexStat(Base):
    __tablename__ = 'sex_stat'

    sex = Column(String(2), primary_key=True)
    ratio = Column(Float(8))


In [8]:
# Merging data to session
for i in range(0, len(last_name)):
    session.merge(
        KoreanLastname(
            lastname=last_name.loc[i, '이름'],
            ratio=round(last_name.loc[i, '비율'], 4)
        )
    )

for i in range(0, len(first_name)):
    session.merge(
        KoreanFirstname(
            firstname=first_name.loc[i, '이름'],
            ratio=round(first_name.loc[i, '비율'], 4)
        )
    )

for i in range(0, len(age_stat)):
    session.merge(
        AgeStat(
            age=age_stat.loc[i, '나이'],
            ratio=round(age_stat.loc[i, '비율'], 4)
        )
    )

for i in range(0, len(region_stat)):
    session.merge(
        RegionStat(
            region=region_stat.loc[i, '지역'],
            ratio=round(region_stat.loc[i, '비율'], 4)
        )
    )

for i in range(0, len(sex_stat)):
    session.merge(
        SexStat(
            sex=sex_stat.loc[i, '성'],
            ratio=round(sex_stat.loc[i, '비율'], 4)
        )
    )


In [9]:
# Flushing session
session.flush()


In [10]:
# Committing session
session.commit()


In [11]:
# Closing session
session.close()
